In [6]:
import googleapiclient.discovery
import googleapiclient.errors
from datetime import datetime
import pandas as pd
import json

In [7]:
channel_id = "UC45i13dEfEVac2IEJT_Nr5Q"

In [8]:
with open('gworkspace_api_key.ini', 'r', encoding='utf-8-sig') as f:
    api_key = f.read()

In [9]:
def getPlaylistVideosIDs(api_key, channel_id):
    api_service_name = "youtube"
    api_version = "v3"
    # Get credentials and create an API client
    youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=api_key)
    request = youtube.channels().list(part="contentDetails", id=channel_id)
    response = request.execute()
    # Response for channel List
    if not response:
        return
    totalResults = response['pageInfo'].get('totalResults', None)
    if not totalResults:
        return
    if totalResults != 1:
        print('Need further check because totalResults is grater than 1.')
        return
    # Get video Lists from playlist_id
    playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    title = None
    nextPageToken = True
    videos_IDs = []
    while nextPageToken:
        if isinstance(nextPageToken, bool):
            request = youtube.playlistItems().list(part='snippet', playlistId=playlist_id, maxResults=50)
        else:
            request = youtube.playlistItems().list(pageToken=nextPageToken, part='snippet', playlistId=playlist_id,
                                                   maxResults=50)
        response = request.execute()
        if not response:
            print('Requesting youtube.playlistItems.list has no response. Exit the loop.')
            break
        nextPageToken = response.get('nextPageToken', None)
        for count, item in enumerate(response['items']):
            if not title:
                title = item['snippet'].get('channelTitle', None)
            if not title:
                title = item['snippet'].get('videoOwnerChannelTitle', None)
            video_item = {
                'title': item['snippet']['title'],
                'videoId': 'https://www.youtube.com/watch?v=' + item['snippet']['resourceId']['videoId'],
                'publishedAt': item['snippet']['publishedAt']
            }
            if 'snippet' in item and 'thumbnails' in item['snippet'] and item['snippet']['thumbnails'].items():
                thumbnails = []
                for style, details in item['snippet']['thumbnails'].items():
                    if details.get('url', None):
                        thumbnails.append(details['url'])
                if thumbnails:
                    video_item['thumbnails'] = thumbnails
            videos_IDs.append(video_item)
    return videos_IDs, title

In [10]:
videos_IDs, title = getPlaylistVideosIDs(api_key=api_key, channel_id=channel_id)
print('title: ', title)
print('total [%s] videos_IDs: %s' % (len(videos_IDs), str(videos_IDs[0])))
print('columns(keys) in videos_IDs: ', videos_IDs[0].keys())

title:  柴鼠兄弟 ZRBros
total [486] videos_IDs: {'title': '護國神山過前高 金融股便當今年績效30%！季配ETF的夢幻填息天數 | 柴鼠國民ETF人氣榜 [2021年9月號]', 'videoId': 'https://www.youtube.com/watch?v=FkKYcs2EhfQ', 'publishedAt': '2021-09-05T14:43:03Z', 'thumbnails': ['https://i.ytimg.com/vi/FkKYcs2EhfQ/default.jpg', 'https://i.ytimg.com/vi/FkKYcs2EhfQ/mqdefault.jpg', 'https://i.ytimg.com/vi/FkKYcs2EhfQ/hqdefault.jpg']}
columns(keys) in videos_IDs:  dict_keys(['title', 'videoId', 'publishedAt', 'thumbnails'])


In [11]:
df = pd.DataFrame(videos_IDs)
df

,title,videoId,publishedAt,thumbnails
0,護國神山過前高 金融股便當今年績效30%！季配ETF的夢幻填息天數 | 柴鼠國民ETF人氣榜...,https://www.youtube.com/watch?v=FkKYcs2EhfQ,2021-09-05T14:43:03Z,[https://i.ytimg.com/vi/FkKYcs2EhfQ/default.jp...
1,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,2021-08-31T04:36:14Z,[https://i.ytimg.com/vi/az2JNgoF6j0/default.jp...
2,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？ | 夯翻鼠FQ68 股票投資,https://www.youtube.com/watch?v=_jNBy07H9nQ,2021-08-26T11:39:55Z,[https://i.ytimg.com/vi/_jNBy07H9nQ/default.jp...
3,00895未來車不只電動車 自駕/共享/儲能/聯運 這改變可能會超越智慧手機 | 柴鼠ETF任意門,https://www.youtube.com/watch?v=Nu-kIfLpw0I,2021-08-23T09:13:00Z,[https://i.ytimg.com/vi/Nu-kIfLpw0I/default.jp...
4,去哪都適用的30個必備職場技能 學起來受用一輩子『你不只是新人，你是好手』｜夯翻鼠聽書S1E6,https://www.youtube.com/watch?v=XYv7iA0_MS0,2021-08-18T00:35:43Z,[https://i.ytimg.com/vi/XYv7iA0_MS0/default.jp...
...,...,...,...,...
481,激賞! 7-11霍格滑茲水果冰球 | 柴鼠開吃,https://www.youtube.com/watch?v=oeWEMQLAHF8,2017-04-12T13:24:17Z,[https://i.ytimg.com/vi/oeWEMQLAHF8/default.jp...
482,第一次開箱就失手 | 柴鼠拆盒子 | DJI Osmo Mobile unbox | 柴鼠開吃,https://www.youtube.com/watch?v=3Wg_3W9VVng,2017-04-07T08:59:55Z,[https://i.ytimg.com/vi/3Wg_3W9VVng/default.jp...
483,暗黑心理學-用漫畫教你控制人心(熱門暢銷書) | 夯翻鼠說書1,https://www.youtube.com/watch?v=IK4dcukFV28,2017-04-07T15:05:54Z,[https://i.ytimg.com/vi/IK4dcukFV28/default.jp...
484,我們想做的YouTube類型居然是...,https://www.youtube.com/watch?v=8YvmzT5m8E4,2017-04-05T09:11:57Z,[https://i.ytimg.com/vi/8YvmzT5m8E4/default.jp...


In [12]:
df.index

RangeIndex(start=0, stop=486, step=1)

In [13]:
type(df['title'])

pandas.core.series.Series

In [14]:
df.columns

Index(['title', 'videoId', 'publishedAt', 'thumbnails'], dtype='object')

In [15]:
df['title']

0      護國神山過前高 金融股便當今年績效30%！季配ETF的夢幻填息天數 | 柴鼠國民ETF人氣榜...
1      最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...
2          圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？ | 夯翻鼠FQ68 股票投資
3      00895未來車不只電動車 自駕/共享/儲能/聯運 這改變可能會超越智慧手機 | 柴鼠ETF任意門
4        去哪都適用的30個必備職場技能 學起來受用一輩子『你不只是新人，你是好手』｜夯翻鼠聽書S1E6
                             ...                        
481                              激賞! 7-11霍格滑茲水果冰球 | 柴鼠開吃
482      第一次開箱就失手 | 柴鼠拆盒子 | DJI Osmo Mobile unbox | 柴鼠開吃
483                      暗黑心理學-用漫畫教你控制人心(熱門暢銷書) | 夯翻鼠說書1
484                                 我們想做的YouTube類型居然是...
485                                              我們是柴鼠兄弟
Name: title, Length: 486, dtype: object

In [17]:
print(df.dtypes)
for i in df.columns:
    print(type(df[i][0]))

title          object
videoId        object
publishedAt    object
thumbnails     object
dtype: object
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>


In [18]:
myfilter = ~df['title'].str.contains(r'\｜|\||\||\│', regex=True)


In [19]:
df[myfilter]

,title,videoId,publishedAt,thumbnails
194,5年100萬頭期款怎麼存？每月1萬5看看AI怎麼算 (ft.國泰世華智能投資),https://www.youtube.com/watch?v=0U-Dflp9cJ8,2019-04-16T03:42:18Z,[https://i.ytimg.com/vi/0U-Dflp9cJ8/default.jp...
217,YouTube年終聚會~創作者們廚藝大PK,https://www.youtube.com/watch?v=iVmGyofMUhI,2018-12-02T14:41:59Z,[https://i.ytimg.com/vi/iVmGyofMUhI/default.jp...
428,世大運比賽場館在哪裡？柴鼠跳給你看(feat Ketakuma激動熊) 2017 Summe...,https://www.youtube.com/watch?v=V2znson_-uw,2017-08-10T18:44:15Z,[https://i.ytimg.com/vi/V2znson_-uw/default.jp...
450,小S吃吃的愛-比比內內點名鯉魚歌繞口令挑戰～紅鯉魚與綠鯉魚(柴鼠點名: Erica大耳 An...,https://www.youtube.com/watch?v=uFfBv3yAwxA,2017-06-10T19:01:52Z,[https://i.ytimg.com/vi/uFfBv3yAwxA/default.jp...
461,YouTuber玩台灣(下集) 晴光市場流淚甜點 (feat AnsonTV台灣美食旅遊頻道 ),https://www.youtube.com/watch?v=39mbWtTUugo,2017-05-21T08:07:42Z,[https://i.ytimg.com/vi/39mbWtTUugo/default.jp...
479,神還原 不要生氣讚美操,https://www.youtube.com/watch?v=lmmRhpABo9o,2017-04-14T10:32:27Z,[https://i.ytimg.com/vi/lmmRhpABo9o/default.jp...
484,我們想做的YouTube類型居然是...,https://www.youtube.com/watch?v=8YvmzT5m8E4,2017-04-05T09:11:57Z,[https://i.ytimg.com/vi/8YvmzT5m8E4/default.jp...
485,我們是柴鼠兄弟,https://www.youtube.com/watch?v=TCWZs8qgoPE,2017-04-04T17:49:21Z,[https://i.ytimg.com/vi/TCWZs8qgoPE/default.jp...


In [20]:
index_list = df[myfilter].index
index_list

Int64Index([194, 217, 428, 450, 461, 479, 484, 485], dtype='int64')

In [21]:
df.iloc[index_list[0]]['title']

'5年100萬頭期款怎麼存？每月1萬5看看AI怎麼算 (ft.國泰世華智能投資)'

In [22]:
for i, j in df[myfilter].iterrows():
    df.loc[i]['title'] = df.loc[i]['title'] + '|沒有分類'
df.loc[index_list]

,title,videoId,publishedAt,thumbnails
194,5年100萬頭期款怎麼存？每月1萬5看看AI怎麼算 (ft.國泰世華智能投資)|沒有分類,https://www.youtube.com/watch?v=0U-Dflp9cJ8,2019-04-16T03:42:18Z,[https://i.ytimg.com/vi/0U-Dflp9cJ8/default.jp...
217,YouTube年終聚會~創作者們廚藝大PK|沒有分類,https://www.youtube.com/watch?v=iVmGyofMUhI,2018-12-02T14:41:59Z,[https://i.ytimg.com/vi/iVmGyofMUhI/default.jp...
428,世大運比賽場館在哪裡？柴鼠跳給你看(feat Ketakuma激動熊) 2017 Summe...,https://www.youtube.com/watch?v=V2znson_-uw,2017-08-10T18:44:15Z,[https://i.ytimg.com/vi/V2znson_-uw/default.jp...
450,小S吃吃的愛-比比內內點名鯉魚歌繞口令挑戰～紅鯉魚與綠鯉魚(柴鼠點名: Erica大耳 An...,https://www.youtube.com/watch?v=uFfBv3yAwxA,2017-06-10T19:01:52Z,[https://i.ytimg.com/vi/uFfBv3yAwxA/default.jp...
461,YouTuber玩台灣(下集) 晴光市場流淚甜點 (feat AnsonTV台灣美食旅遊頻道...,https://www.youtube.com/watch?v=39mbWtTUugo,2017-05-21T08:07:42Z,[https://i.ytimg.com/vi/39mbWtTUugo/default.jp...
479,神還原 不要生氣讚美操|沒有分類,https://www.youtube.com/watch?v=lmmRhpABo9o,2017-04-14T10:32:27Z,[https://i.ytimg.com/vi/lmmRhpABo9o/default.jp...
484,我們想做的YouTube類型居然是...|沒有分類,https://www.youtube.com/watch?v=8YvmzT5m8E4,2017-04-05T09:11:57Z,[https://i.ytimg.com/vi/8YvmzT5m8E4/default.jp...
485,我們是柴鼠兄弟|沒有分類,https://www.youtube.com/watch?v=TCWZs8qgoPE,2017-04-04T17:49:21Z,[https://i.ytimg.com/vi/TCWZs8qgoPE/default.jp...


In [23]:
df[['title','category']] = df['title'].str.split(pat=r'\｜|\||\||\│', n=1, expand=True)
df

,title,videoId,publishedAt,thumbnails,category
0,護國神山過前高 金融股便當今年績效30%！季配ETF的夢幻填息天數,https://www.youtube.com/watch?v=FkKYcs2EhfQ,2021-09-05T14:43:03Z,[https://i.ytimg.com/vi/FkKYcs2EhfQ/default.jp...,柴鼠國民ETF人氣榜 [2021年9月號]
1,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,2021-08-31T04:36:14Z,[https://i.ytimg.com/vi/az2JNgoF6j0/default.jp...,夯翻鼠FQ&A48
2,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？,https://www.youtube.com/watch?v=_jNBy07H9nQ,2021-08-26T11:39:55Z,[https://i.ytimg.com/vi/_jNBy07H9nQ/default.jp...,夯翻鼠FQ68 股票投資
3,00895未來車不只電動車 自駕/共享/儲能/聯運 這改變可能會超越智慧手機,https://www.youtube.com/watch?v=Nu-kIfLpw0I,2021-08-23T09:13:00Z,[https://i.ytimg.com/vi/Nu-kIfLpw0I/default.jp...,柴鼠ETF任意門
4,去哪都適用的30個必備職場技能 學起來受用一輩子『你不只是新人，你是好手』,https://www.youtube.com/watch?v=XYv7iA0_MS0,2021-08-18T00:35:43Z,[https://i.ytimg.com/vi/XYv7iA0_MS0/default.jp...,夯翻鼠聽書S1E6
...,...,...,...,...,...
481,激賞! 7-11霍格滑茲水果冰球,https://www.youtube.com/watch?v=oeWEMQLAHF8,2017-04-12T13:24:17Z,[https://i.ytimg.com/vi/oeWEMQLAHF8/default.jp...,柴鼠開吃
482,第一次開箱就失手,https://www.youtube.com/watch?v=3Wg_3W9VVng,2017-04-07T08:59:55Z,[https://i.ytimg.com/vi/3Wg_3W9VVng/default.jp...,柴鼠拆盒子 | DJI Osmo Mobile unbox | 柴鼠開吃
483,暗黑心理學-用漫畫教你控制人心(熱門暢銷書),https://www.youtube.com/watch?v=IK4dcukFV28,2017-04-07T15:05:54Z,[https://i.ytimg.com/vi/IK4dcukFV28/default.jp...,夯翻鼠說書1
484,我們想做的YouTube類型居然是...,https://www.youtube.com/watch?v=8YvmzT5m8E4,2017-04-05T09:11:57Z,[https://i.ytimg.com/vi/8YvmzT5m8E4/default.jp...,沒有分類


In [24]:
df[df.category.isna()]
# index_list = df[df.category.isna()].index

,title,videoId,publishedAt,thumbnails,category


In [25]:
df['thumbnails'].loc[0]

['https://i.ytimg.com/vi/FkKYcs2EhfQ/default.jpg',
 'https://i.ytimg.com/vi/FkKYcs2EhfQ/mqdefault.jpg',
 'https://i.ytimg.com/vi/FkKYcs2EhfQ/hqdefault.jpg']

In [26]:
type(df['thumbnails'].loc[0])

list

In [27]:
#df['thumbnails'] = df['thumbnails'].apply(lambda lt: lt[-1])
def get_last(column_list):
    return column_list[-1]

In [28]:
df['thumbnails'] = df['thumbnails'].apply(get_last)
df

,title,videoId,publishedAt,thumbnails,category
0,護國神山過前高 金融股便當今年績效30%！季配ETF的夢幻填息天數,https://www.youtube.com/watch?v=FkKYcs2EhfQ,2021-09-05T14:43:03Z,https://i.ytimg.com/vi/FkKYcs2EhfQ/hqdefault.jpg,柴鼠國民ETF人氣榜 [2021年9月號]
1,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,2021-08-31T04:36:14Z,https://i.ytimg.com/vi/az2JNgoF6j0/hqdefault.jpg,夯翻鼠FQ&A48
2,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？,https://www.youtube.com/watch?v=_jNBy07H9nQ,2021-08-26T11:39:55Z,https://i.ytimg.com/vi/_jNBy07H9nQ/hqdefault.jpg,夯翻鼠FQ68 股票投資
3,00895未來車不只電動車 自駕/共享/儲能/聯運 這改變可能會超越智慧手機,https://www.youtube.com/watch?v=Nu-kIfLpw0I,2021-08-23T09:13:00Z,https://i.ytimg.com/vi/Nu-kIfLpw0I/maxresdefau...,柴鼠ETF任意門
4,去哪都適用的30個必備職場技能 學起來受用一輩子『你不只是新人，你是好手』,https://www.youtube.com/watch?v=XYv7iA0_MS0,2021-08-18T00:35:43Z,https://i.ytimg.com/vi/XYv7iA0_MS0/maxresdefau...,夯翻鼠聽書S1E6
...,...,...,...,...,...
481,激賞! 7-11霍格滑茲水果冰球,https://www.youtube.com/watch?v=oeWEMQLAHF8,2017-04-12T13:24:17Z,https://i.ytimg.com/vi/oeWEMQLAHF8/hqdefault.jpg,柴鼠開吃
482,第一次開箱就失手,https://www.youtube.com/watch?v=3Wg_3W9VVng,2017-04-07T08:59:55Z,https://i.ytimg.com/vi/3Wg_3W9VVng/sddefault.jpg,柴鼠拆盒子 | DJI Osmo Mobile unbox | 柴鼠開吃
483,暗黑心理學-用漫畫教你控制人心(熱門暢銷書),https://www.youtube.com/watch?v=IK4dcukFV28,2017-04-07T15:05:54Z,https://i.ytimg.com/vi/IK4dcukFV28/sddefault.jpg,夯翻鼠說書1
484,我們想做的YouTube類型居然是...,https://www.youtube.com/watch?v=8YvmzT5m8E4,2017-04-05T09:11:57Z,https://i.ytimg.com/vi/8YvmzT5m8E4/maxresdefau...,沒有分類


In [29]:
print(type(df['thumbnails'].loc[0]))

<class 'str'>


In [30]:
fq= df[df['category'].str.contains('FQ')].copy()
fq

,title,videoId,publishedAt,thumbnails,category
1,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,2021-08-31T04:36:14Z,https://i.ytimg.com/vi/az2JNgoF6j0/hqdefault.jpg,夯翻鼠FQ&A48
2,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？,https://www.youtube.com/watch?v=_jNBy07H9nQ,2021-08-26T11:39:55Z,https://i.ytimg.com/vi/_jNBy07H9nQ/hqdefault.jpg,夯翻鼠FQ68 股票投資
11,ETN不是不能買 但好像變成ETF基礎總複習,https://www.youtube.com/watch?v=Y1Pj7Bxbqzs,2021-07-18T13:12:15Z,https://i.ytimg.com/vi/Y1Pj7Bxbqzs/maxresdefau...,夯翻鼠FQ67
13,選利率還是選額度？⟪2021下半年數位帳戶高利活儲PK⟫ 1.1%能簡單就不要複雜,https://www.youtube.com/watch?v=vxbPxC2Obwo,2021-07-09T12:21:51Z,https://i.ytimg.com/vi/vxbPxC2Obwo/maxresdefau...,夯翻鼠FQ&A47 活存 定存
17,0050和0056超級換菜週 法人買賣超失真？ETF的類鎖碼效果,https://www.youtube.com/watch?v=PaJvgJU7p7c,2021-06-29T07:12:54Z,https://i.ytimg.com/vi/PaJvgJU7p7c/maxresdefau...,夯翻鼠FQ&A46
...,...,...,...,...,...
313,不意外！台股平均20年殖利率排行前三名是他們⋯ 放山雞選股法(網站實作篇),https://www.youtube.com/watch?v=cY0P5fh-d9o,2018-01-16T04:42:46Z,https://i.ytimg.com/vi/cY0P5fh-d9o/maxresdefau...,夯翻鼠FQ5 投資理財
314,2018存股怎麼選？新手秒懂的-放山雞選股法(概念篇),https://www.youtube.com/watch?v=854HH85QCdU,2018-01-14T13:52:02Z,https://i.ytimg.com/vi/854HH85QCdU/maxresdefau...,夯翻鼠FQ4 投資理財
340,匯率霧傻傻？先換起來放？錢被銀行默默吃掉的三個秘密,https://www.youtube.com/watch?v=U8wLmL6PKnI,2017-11-27T14:54:56Z,https://i.ytimg.com/vi/U8wLmL6PKnI/sddefault.jpg,夯翻鼠FQ3 投資理財
371,存股一定打敗銀行定存？常被忽略的三個弱點和一個關鍵...,https://www.youtube.com/watch?v=kF6XomSRcUw,2017-10-27T11:37:04Z,https://i.ytimg.com/vi/kF6XomSRcUw/sddefault.jpg,夯翻鼠FQ2 投資理財


In [31]:
fq.sort_values('publishedAt', inplace=True, ascending=False)
fq.reset_index(inplace=True, drop=True)
fq

,title,videoId,publishedAt,thumbnails,category
0,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,2021-08-31T04:36:14Z,https://i.ytimg.com/vi/az2JNgoF6j0/hqdefault.jpg,夯翻鼠FQ&A48
1,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？,https://www.youtube.com/watch?v=_jNBy07H9nQ,2021-08-26T11:39:55Z,https://i.ytimg.com/vi/_jNBy07H9nQ/hqdefault.jpg,夯翻鼠FQ68 股票投資
2,ETN不是不能買 但好像變成ETF基礎總複習,https://www.youtube.com/watch?v=Y1Pj7Bxbqzs,2021-07-18T13:12:15Z,https://i.ytimg.com/vi/Y1Pj7Bxbqzs/maxresdefau...,夯翻鼠FQ67
3,選利率還是選額度？⟪2021下半年數位帳戶高利活儲PK⟫ 1.1%能簡單就不要複雜,https://www.youtube.com/watch?v=vxbPxC2Obwo,2021-07-09T12:21:51Z,https://i.ytimg.com/vi/vxbPxC2Obwo/maxresdefau...,夯翻鼠FQ&A47 活存 定存
4,0050和0056超級換菜週 法人買賣超失真？ETF的類鎖碼效果,https://www.youtube.com/watch?v=PaJvgJU7p7c,2021-06-29T07:12:54Z,https://i.ytimg.com/vi/PaJvgJU7p7c/maxresdefau...,夯翻鼠FQ&A46
...,...,...,...,...,...
112,不意外！台股平均20年殖利率排行前三名是他們⋯ 放山雞選股法(網站實作篇),https://www.youtube.com/watch?v=cY0P5fh-d9o,2018-01-16T04:42:46Z,https://i.ytimg.com/vi/cY0P5fh-d9o/maxresdefau...,夯翻鼠FQ5 投資理財
113,2018存股怎麼選？新手秒懂的-放山雞選股法(概念篇),https://www.youtube.com/watch?v=854HH85QCdU,2018-01-14T13:52:02Z,https://i.ytimg.com/vi/854HH85QCdU/maxresdefau...,夯翻鼠FQ4 投資理財
114,匯率霧傻傻？先換起來放？錢被銀行默默吃掉的三個秘密,https://www.youtube.com/watch?v=U8wLmL6PKnI,2017-11-27T14:54:56Z,https://i.ytimg.com/vi/U8wLmL6PKnI/sddefault.jpg,夯翻鼠FQ3 投資理財
115,存股一定打敗銀行定存？常被忽略的三個弱點和一個關鍵...,https://www.youtube.com/watch?v=kF6XomSRcUw,2017-10-27T11:37:04Z,https://i.ytimg.com/vi/kF6XomSRcUw/sddefault.jpg,夯翻鼠FQ2 投資理財


In [32]:
def get_video_description(video_id):
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)
    request = youtube.videos().list(part="snippet", id=video_id)
    response = request.execute()
    return response['items'][0]['snippet']['description']

In [33]:
fq['description'] = fq['videoId'].str[32:].apply(get_video_description)
fq

,title,videoId,publishedAt,thumbnails,category,description
0,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,2021-08-31T04:36:14Z,https://i.ytimg.com/vi/az2JNgoF6j0/hqdefault.jpg,夯翻鼠FQ&A48,🤓今年4月我們曾經介紹過台股ETF #季配轉月配 的組合，但不管是用87888搭配712或是...
1,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？,https://www.youtube.com/watch?v=_jNBy07H9nQ,2021-08-26T11:39:55Z,https://i.ytimg.com/vi/_jNBy07H9nQ/hqdefault.jpg,夯翻鼠FQ68 股票投資,🤓常常在股市聽到什麼跳空缺口、空方缺口或是誰已經補掉某個缺口，但K線與K線之間為什麼會出現缺...
2,ETN不是不能買 但好像變成ETF基礎總複習,https://www.youtube.com/watch?v=Y1Pj7Bxbqzs,2021-07-18T13:12:15Z,https://i.ytimg.com/vi/Y1Pj7Bxbqzs/maxresdefau...,夯翻鼠FQ67,不知道是不是ETF越來越夯的關係，最近敲碗問ETN的人還真不少，其實也不是不能買，只是買之前...
3,選利率還是選額度？⟪2021下半年數位帳戶高利活儲PK⟫ 1.1%能簡單就不要複雜,https://www.youtube.com/watch?v=vxbPxC2Obwo,2021-07-09T12:21:51Z,https://i.ytimg.com/vi/vxbPxC2Obwo/maxresdefau...,夯翻鼠FQ&A47 活存 定存,來PK2021下半年數位帳戶高利活儲\n✅完整PK大圖 https://bit.ly/3wv...
4,0050和0056超級換菜週 法人買賣超失真？ETF的類鎖碼效果,https://www.youtube.com/watch?v=PaJvgJU7p7c,2021-06-29T07:12:54Z,https://i.ytimg.com/vi/PaJvgJU7p7c/maxresdefau...,夯翻鼠FQ&A46,上禮拜剛結束每半年一次的「ETF超級換菜週🥬」，由於這次納入三大航海王和面板，引起了各方相當...
...,...,...,...,...,...,...
112,不意外！台股平均20年殖利率排行前三名是他們⋯ 放山雞選股法(網站實作篇),https://www.youtube.com/watch?v=cY0P5fh-d9o,2018-01-16T04:42:46Z,https://i.ytimg.com/vi/cY0P5fh-d9o/maxresdefau...,夯翻鼠FQ5 投資理財,看完選股概念篇之後，我們馬上來實作看看🤓\n鼠教你用一個神強大的免費股市資訊網站\n可以秒找...
113,2018存股怎麼選？新手秒懂的-放山雞選股法(概念篇),https://www.youtube.com/watch?v=854HH85QCdU,2018-01-14T13:52:02Z,https://i.ytimg.com/vi/854HH85QCdU/maxresdefau...,夯翻鼠FQ4 投資理財,新的一年開始，準備好的新的理財計畫了嗎？🤓\n繼上次定存和存股的比較，很多人可能都想知道怎麼...
114,匯率霧傻傻？先換起來放？錢被銀行默默吃掉的三個秘密,https://www.youtube.com/watch?v=U8wLmL6PKnI,2017-11-27T14:54:56Z,https://i.ytimg.com/vi/U8wLmL6PKnI/sddefault.jpg,夯翻鼠FQ3 投資理財,要出國了，準備好要換外幣了？🧐\n匯率看板一堆密密麻麻的數字總是有看沒有懂？\n為什麼出國刷...
115,存股一定打敗銀行定存？常被忽略的三個弱點和一個關鍵...,https://www.youtube.com/watch?v=kF6XomSRcUw,2017-10-27T11:37:04Z,https://i.ytimg.com/vi/kF6XomSRcUw/sddefault.jpg,夯翻鼠FQ2 投資理財,銀行定存就是那麼1%\n各種理財商品都喜歡說自己打敗它\n但是存現金真的不如存股票嗎？\n股...


In [34]:
To_remove_lst = [
  '【跟著柴學FQ 做自己的提款機】',
  '⭐️博客來全館即時榜、商業書籍榜、7日/30日排行榜、新書排行榜冠軍',
  '⭐️2020上半年MOMO網路書店暢銷榜Top7', '➤博客來 https://reurl.cc/8GnRXy',
  '➤誠品 https://reurl.cc/Kkj5lq', '➤金石堂 https://reurl.cc/z8zLp6',
  '【不用花錢也可以贊助柴鼠】', 'https://youtu.be/DGV964Fks2s',
  '【訂閱柴鼠也是一種鼓勵】',
  'http://bit.ly/2INZTDI',
  '【柴鼠兄弟相關頻道】',
  'YouTube頻道 https://www.youtube.com/c/柴鼠兄弟ZRBros',
  'FB粉絲頁 https://www.facebook.com/zrbros',
  'IG https://www.instagram.com/zrbros/',
  '【聯絡柴鼠】',
  'e-mail：paike.cat@gmail.com',
  '⭐️出版三個月熱銷突破十刷三萬冊',
  '⭐️博客來全館即時榜、商業書籍榜、7日/30日排行榜、新書排行榜冠軍',
  '⭐️博客來2020上半年暢銷書籍榜Top10',
  '⭐️2020上半年MOMO網路書店暢銷榜Top7',
  '【柴鼠LINE貼圖】',
  'https://pse.is/LRUBU',
  '⭐️博客來全館即時榜、商業書籍榜、7日/30日暢銷榜、新書排行榜冠軍',
  '⭐️2020上半年MOMO網路書店暢銷榜Top7',
  '🍎博客來',
  'https://reurl.cc/8GnRXy',
  '🍎誠品',
  'https://reurl.cc/Kkj5lq',
  '🍎金石堂',
  'https://reurl.cc/z8zLp6',
  '⭐️2020誠品書店年度暢銷Top2',
  '⭐️博客來2020年度百大暢銷書榜Top8',
  '⭐️博客來2020年度暢銷書榜Top8',
  '⭐️連續12週\(4/2~6/24\)蟬聯誠品書店財經/商業暢銷榜冠軍',
  'bilibili https://space.bilibili.com/130090309',
  '愛奇藝 http://tw.iqiyi.com/u/1421309132',
  'https://ppt.cc/fUrThx'
  ]

In [35]:
fq['description'] = fq['description'].str.replace('|'.join(To_remove_lst), '', regex=True)
fq

,title,videoId,publishedAt,thumbnails,category,description
0,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,2021-08-31T04:36:14Z,https://i.ytimg.com/vi/az2JNgoF6j0/hqdefault.jpg,夯翻鼠FQ&A48,🤓今年4月我們曾經介紹過台股ETF #季配轉月配 的組合，但不管是用87888搭配712或是...
1,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？,https://www.youtube.com/watch?v=_jNBy07H9nQ,2021-08-26T11:39:55Z,https://i.ytimg.com/vi/_jNBy07H9nQ/hqdefault.jpg,夯翻鼠FQ68 股票投資,🤓常常在股市聽到什麼跳空缺口、空方缺口或是誰已經補掉某個缺口，但K線與K線之間為什麼會出現缺...
2,ETN不是不能買 但好像變成ETF基礎總複習,https://www.youtube.com/watch?v=Y1Pj7Bxbqzs,2021-07-18T13:12:15Z,https://i.ytimg.com/vi/Y1Pj7Bxbqzs/maxresdefau...,夯翻鼠FQ67,不知道是不是ETF越來越夯的關係，最近敲碗問ETN的人還真不少，其實也不是不能買，只是買之前...
3,選利率還是選額度？⟪2021下半年數位帳戶高利活儲PK⟫ 1.1%能簡單就不要複雜,https://www.youtube.com/watch?v=vxbPxC2Obwo,2021-07-09T12:21:51Z,https://i.ytimg.com/vi/vxbPxC2Obwo/maxresdefau...,夯翻鼠FQ&A47 活存 定存,來PK2021下半年數位帳戶高利活儲\n✅完整PK大圖 https://bit.ly/3wv...
4,0050和0056超級換菜週 法人買賣超失真？ETF的類鎖碼效果,https://www.youtube.com/watch?v=PaJvgJU7p7c,2021-06-29T07:12:54Z,https://i.ytimg.com/vi/PaJvgJU7p7c/maxresdefau...,夯翻鼠FQ&A46,上禮拜剛結束每半年一次的「ETF超級換菜週🥬」，由於這次納入三大航海王和面板，引起了各方相當...
...,...,...,...,...,...,...
112,不意外！台股平均20年殖利率排行前三名是他們⋯ 放山雞選股法(網站實作篇),https://www.youtube.com/watch?v=cY0P5fh-d9o,2018-01-16T04:42:46Z,https://i.ytimg.com/vi/cY0P5fh-d9o/maxresdefau...,夯翻鼠FQ5 投資理財,看完選股概念篇之後，我們馬上來實作看看🤓\n鼠教你用一個神強大的免費股市資訊網站\n可以秒找...
113,2018存股怎麼選？新手秒懂的-放山雞選股法(概念篇),https://www.youtube.com/watch?v=854HH85QCdU,2018-01-14T13:52:02Z,https://i.ytimg.com/vi/854HH85QCdU/maxresdefau...,夯翻鼠FQ4 投資理財,新的一年開始，準備好的新的理財計畫了嗎？🤓\n繼上次定存和存股的比較，很多人可能都想知道怎麼...
114,匯率霧傻傻？先換起來放？錢被銀行默默吃掉的三個秘密,https://www.youtube.com/watch?v=U8wLmL6PKnI,2017-11-27T14:54:56Z,https://i.ytimg.com/vi/U8wLmL6PKnI/sddefault.jpg,夯翻鼠FQ3 投資理財,要出國了，準備好要換外幣了？🧐\n匯率看板一堆密密麻麻的數字總是有看沒有懂？\n為什麼出國刷...
115,存股一定打敗銀行定存？常被忽略的三個弱點和一個關鍵...,https://www.youtube.com/watch?v=kF6XomSRcUw,2017-10-27T11:37:04Z,https://i.ytimg.com/vi/kF6XomSRcUw/sddefault.jpg,夯翻鼠FQ2 投資理財,銀行定存就是那麼1%\n各種理財商品都喜歡說自己打敗它\n但是存現金真的不如存股票嗎？\n股...


In [58]:
fq[fq['title'].str.contains('ETN不是不能買 但好像變成ETF基礎總複習')].description.to_list()[0]


'不知道是不是ETF越來越夯的關係，最近敲碗問ETN的人還真不少，其實也不是不能買，只是買之前需要準備比ETF更進階的商品知識，所以柴鼠這次又一鼓作氣K了一堆公開說明書，把台股目前30檔ETN商品的基本資料建立起來。這一集我們就來深入研究這個跟ETF有點像，但又不一樣的新領域。\n✅30檔ETN資料表下載 https://bit.ly/2Uq9VoT\n\n#整集做完覺得比像ETF基礎總複習🤣\n#極限和期限\n#老闆有ET👽\n#資料型YouTuber\n\n0:00 ETN不是不能買\n0:40 投信和證券差在哪裡？\n2:12 都有ET但結構不同\n4:39 ETF為什麼有追蹤誤差\n6:24 券商怎麼避險？\n8:26 ETN最大優勢\n9:25 三劑ETN疫苗\n11:07 FQQ時間\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [59]:
fq['description'] = fq['description'].str.replace(r"(https?://[-A-Za-z0-9+@#/%?=~_|!:,.;]*[-A-Za-z0-9+@#/%=~_|])", r'<a href="\1" target="_blank">\1</a>', regex=True)
fq[fq['title'].str.contains('ETN不是不能買 但好像變成ETF基礎總複習')].description.to_list()[0]

'不知道是不是ETF越來越夯的關係，最近敲碗問ETN的人還真不少，其實也不是不能買，只是買之前需要準備比ETF更進階的商品知識，所以柴鼠這次又一鼓作氣K了一堆公開說明書，把台股目前30檔ETN商品的基本資料建立起來。這一集我們就來深入研究這個跟ETF有點像，但又不一樣的新領域。\n✅30檔ETN資料表下載 <a href="https://bit.ly/2Uq9VoT" target="_blank">https://bit.ly/2Uq9VoT</a>\n\n#整集做完覺得比像ETF基礎總複習🤣\n#極限和期限\n#老闆有ET👽\n#資料型YouTuber\n\n0:00 ETN不是不能買\n0:40 投信和證券差在哪裡？\n2:12 都有ET但結構不同\n4:39 ETF為什麼有追蹤誤差\n6:24 券商怎麼避險？\n8:26 ETN最大優勢\n9:25 三劑ETN疫苗\n11:07 FQQ時間\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [60]:
fq['description'] = fq['description'].str.strip()
fq['description'] = fq['description'].str.replace(r"\n{3,}", '', regex=True)
fq['description'] = fq['description'].str.replace(r"\n", '<br>', regex=True)

In [61]:
fq['publishedAt'] = fq['publishedAt'].apply(lambda x: datetime.fromisoformat(x[:-1]).strftime("%b-%u-%Y"))
fq

,title,videoId,publishedAt,thumbnails,category,description
0,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,Aug-2-2021,https://i.ytimg.com/vi/az2JNgoF6j0/hqdefault.jpg,夯翻鼠FQ&A48,🤓今年4月我們曾經介紹過台股ETF #季配轉月配 的組合，但不管是用87888搭配712或是...
1,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？,https://www.youtube.com/watch?v=_jNBy07H9nQ,Aug-4-2021,https://i.ytimg.com/vi/_jNBy07H9nQ/hqdefault.jpg,夯翻鼠FQ68 股票投資,🤓常常在股市聽到什麼跳空缺口、空方缺口或是誰已經補掉某個缺口，但K線與K線之間為什麼會出現缺...
2,ETN不是不能買 但好像變成ETF基礎總複習,https://www.youtube.com/watch?v=Y1Pj7Bxbqzs,Jul-7-2021,https://i.ytimg.com/vi/Y1Pj7Bxbqzs/maxresdefau...,夯翻鼠FQ67,不知道是不是ETF越來越夯的關係，最近敲碗問ETN的人還真不少，其實也不是不能買，只是買之前...
3,選利率還是選額度？⟪2021下半年數位帳戶高利活儲PK⟫ 1.1%能簡單就不要複雜,https://www.youtube.com/watch?v=vxbPxC2Obwo,Jul-5-2021,https://i.ytimg.com/vi/vxbPxC2Obwo/maxresdefau...,夯翻鼠FQ&A47 活存 定存,"來PK2021下半年數位帳戶高利活儲<br>✅完整PK大圖 <a href=""https:/..."
4,0050和0056超級換菜週 法人買賣超失真？ETF的類鎖碼效果,https://www.youtube.com/watch?v=PaJvgJU7p7c,Jun-2-2021,https://i.ytimg.com/vi/PaJvgJU7p7c/maxresdefau...,夯翻鼠FQ&A46,上禮拜剛結束每半年一次的「ETF超級換菜週🥬」，由於這次納入三大航海王和面板，引起了各方相當...
...,...,...,...,...,...,...
112,不意外！台股平均20年殖利率排行前三名是他們⋯ 放山雞選股法(網站實作篇),https://www.youtube.com/watch?v=cY0P5fh-d9o,Jan-2-2018,https://i.ytimg.com/vi/cY0P5fh-d9o/maxresdefau...,夯翻鼠FQ5 投資理財,看完選股概念篇之後，我們馬上來實作看看🤓<br>鼠教你用一個神強大的免費股市資訊網站<br>...
113,2018存股怎麼選？新手秒懂的-放山雞選股法(概念篇),https://www.youtube.com/watch?v=854HH85QCdU,Jan-7-2018,https://i.ytimg.com/vi/854HH85QCdU/maxresdefau...,夯翻鼠FQ4 投資理財,新的一年開始，準備好的新的理財計畫了嗎？🤓<br>繼上次定存和存股的比較，很多人可能都想知道...
114,匯率霧傻傻？先換起來放？錢被銀行默默吃掉的三個秘密,https://www.youtube.com/watch?v=U8wLmL6PKnI,Nov-1-2017,https://i.ytimg.com/vi/U8wLmL6PKnI/sddefault.jpg,夯翻鼠FQ3 投資理財,要出國了，準備好要換外幣了？🧐<br>匯率看板一堆密密麻麻的數字總是有看沒有懂？<br>為什...
115,存股一定打敗銀行定存？常被忽略的三個弱點和一個關鍵...,https://www.youtube.com/watch?v=kF6XomSRcUw,Oct-5-2017,https://i.ytimg.com/vi/kF6XomSRcUw/sddefault.jpg,夯翻鼠FQ2 投資理財,銀行定存就是那麼1%<br>各種理財商品都喜歡說自己打敗它<br>但是存現金真的不如存股票嗎...


In [62]:
fq.index += 1
fq.reset_index(inplace=True, drop=False)
fq.rename(columns={'index': '序', 'title': '標題', 'videoId': '網址', 'publishedAt': '發佈日期', 'thumbnails': '封面圖片', 'category': '分類', 'description': '影片說明', 'tags': '標籤', 'categoryId': '號'}, inplace=True)
fq.set_index('發佈日期', inplace=True)
fq

,序,標題,網址,封面圖片,分類,影片說明
發佈日期,,,,,,
Aug-2-2021,1,最新ETF月配組合8788896！配息月曆2.0大改版 00878 00888 00896超...,https://www.youtube.com/watch?v=az2JNgoF6j0,https://i.ytimg.com/vi/az2JNgoF6j0/hqdefault.jpg,夯翻鼠FQ&A48,🤓今年4月我們曾經介紹過台股ETF #季配轉月配 的組合，但不管是用87888搭配712或是...
Aug-4-2021,2,圖解技術面跳空缺口 怎麼又出現神秘數字「3」是氣墊還是氣壓？,https://www.youtube.com/watch?v=_jNBy07H9nQ,https://i.ytimg.com/vi/_jNBy07H9nQ/hqdefault.jpg,夯翻鼠FQ68 股票投資,🤓常常在股市聽到什麼跳空缺口、空方缺口或是誰已經補掉某個缺口，但K線與K線之間為什麼會出現缺...
Jul-7-2021,3,ETN不是不能買 但好像變成ETF基礎總複習,https://www.youtube.com/watch?v=Y1Pj7Bxbqzs,https://i.ytimg.com/vi/Y1Pj7Bxbqzs/maxresdefau...,夯翻鼠FQ67,不知道是不是ETF越來越夯的關係，最近敲碗問ETN的人還真不少，其實也不是不能買，只是買之前...
Jul-5-2021,4,選利率還是選額度？⟪2021下半年數位帳戶高利活儲PK⟫ 1.1%能簡單就不要複雜,https://www.youtube.com/watch?v=vxbPxC2Obwo,https://i.ytimg.com/vi/vxbPxC2Obwo/maxresdefau...,夯翻鼠FQ&A47 活存 定存,"來PK2021下半年數位帳戶高利活儲<br>✅完整PK大圖 <a href=""https:/..."
Jun-2-2021,5,0050和0056超級換菜週 法人買賣超失真？ETF的類鎖碼效果,https://www.youtube.com/watch?v=PaJvgJU7p7c,https://i.ytimg.com/vi/PaJvgJU7p7c/maxresdefau...,夯翻鼠FQ&A46,上禮拜剛結束每半年一次的「ETF超級換菜週🥬」，由於這次納入三大航海王和面板，引起了各方相當...
...,...,...,...,...,...,...
Jan-2-2018,113,不意外！台股平均20年殖利率排行前三名是他們⋯ 放山雞選股法(網站實作篇),https://www.youtube.com/watch?v=cY0P5fh-d9o,https://i.ytimg.com/vi/cY0P5fh-d9o/maxresdefau...,夯翻鼠FQ5 投資理財,看完選股概念篇之後，我們馬上來實作看看🤓<br>鼠教你用一個神強大的免費股市資訊網站<br>...
Jan-7-2018,114,2018存股怎麼選？新手秒懂的-放山雞選股法(概念篇),https://www.youtube.com/watch?v=854HH85QCdU,https://i.ytimg.com/vi/854HH85QCdU/maxresdefau...,夯翻鼠FQ4 投資理財,新的一年開始，準備好的新的理財計畫了嗎？🤓<br>繼上次定存和存股的比較，很多人可能都想知道...
Nov-1-2017,115,匯率霧傻傻？先換起來放？錢被銀行默默吃掉的三個秘密,https://www.youtube.com/watch?v=U8wLmL6PKnI,https://i.ytimg.com/vi/U8wLmL6PKnI/sddefault.jpg,夯翻鼠FQ3 投資理財,要出國了，準備好要換外幣了？🧐<br>匯率看板一堆密密麻麻的數字總是有看沒有懂？<br>為什...


In [63]:
html_string = '''
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>HTML Pandas Dataframe with CSS</title>
    <style>
    .mystyle {{
        font-size: 11pt;
        font-family: Arial;
        border-collapse: collapse;
        border: 1px solid silver;
    }}

    .mystyle th {{
        background: silver;
        padding: 5px;
    }}

    .mystyle td {{
        padding: 5px;
    }}

    .mystyle tr:nth-child(even) {{
        background: #E0E0E0;
    }}

    .mystyle tr:hover {{
        background: silver;
        cursor: pointer;
    }}
    </style>
    </head>
    <body>
        {table}
    </body>
</html>'''

In [64]:
with open('%s.html' % title, 'w', encoding='utf-8-sig') as f:
    f.write(html_string.format(table=fq.to_html(classes='mystyle', index_names=False, render_links=True, escape=False)))
print('檔案 [%s.html] 已存檔完成。' % title)

檔案 [柴鼠兄弟 ZRBros.html] 已存檔完成。
